In [1]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#
# Much appreciation to the Pymoo project for providing the optimization framework used herein:
#
# pymoo: Multi-objective Optimization in Python
# https://github.com/anyoptimization/pymoo
# https://pymoo.org/index.html
#



Computation of TL impedance transformation (lossless, lossy Telegrapher's Equations).
TL length specified in degrees at center of band of interest; properly corrected for electrical length at other frequencies.


In [76]:

# Precompute some arrays
import numpy as np
from numba import njit

# Precompute arrays A,B,C,D to speed impedance transformation calculations (lossless TL)
#   zoa     z of first matching section (connected to antenna)
#   zob     z of second matching section
#   flow, fhigh, nfreq      freq band of interest
#
#   Usage:  precompute A,B,C,D once for specified parameters
#       A,B,C,D = series_match_precompute(zoa=50,zob=75,nfreq=9,flow=3.5,fhigh=4.0)
#
#       z = (zs*A[a,b] + B[a,b]) / ((1j)*zs*C[a,b] + D[a,b])
#           where   zs is a row vector of complex zs, shape (1,nfreq)
#                   a,b   lengths of zoa,zob matching section (degrees)
#
@njit
def series_match_precompute(zoa=50,zob=75,nfreq=9,flow=3.5,fhigh=4.0):
    # Scale phase delay for freqs across band of interest
    phscale = (np.linspace(flow,fhigh,num=nfreq) / ((fhigh+flow)/2))[None,:]
    A = np.empty((181,181,nfreq))
    B = np.empty((181,181,nfreq), dtype=np.complex128)
    C = np.empty((181,181,nfreq))
    D = np.empty((181,181,nfreq))
    for a in range(181):
        tana = np.tan(np.deg2rad(a*phscale))
        for b in range(181):
            tanb = np.tan(np.deg2rad(b*phscale))
            A[a,b] = zoa*zob - zob**2 * tana * tanb
            B[a,b] = zoa*zob*(zoa*tana + zob*tanb)*(1j)
            C[a,b] = zob*tana + zoa*tanb
            D[a,b] = zoa*(zob - zoa * tana * tanb)
    return A,B,C,D
# Exec time 35ms


# Scan combinations of matching-section lengths to find optimum vswr in band of interest
#
# Args:
#   zs      row vector of complex zs, shape (1,nfreq)
#   step    step size (degrees) of matching section lengths,
#           e.g. if step=2 all combinations of a=(0,2,4,...) and b=(0,2,4,...) will be tried
#
# Returns:  optimal lengths of matching sections zoa,zob for minimum vswr in freq band
#
@njit
def series_match_scan(zs, A,B,C,D, step=1, z0=50.0):
    aopt, bopt, vswr_max_opt, vswr_curve_opt = (0, 0, 99999.0, None)
    for a in range(0,A.shape[0],step):
        for b in range(0,A.shape[1],step):

            z = (zs*A[a,b] + B[a,b]) / ((1j)*zs*C[a,b] + D[a,b])
            arf = np.abs((z - z0) / (z + z0))         # Reflection coefs
            vswr_curve = (1 + arf) / (1 - arf)
            vswr_max = np.max(vswr_curve)
            if vswr_max < vswr_max_opt:
                vswr_max_opt = vswr_max
                aopt = a
                bopt = b
                vswr_curve_opt = vswr_curve
    return aopt,bopt,vswr_curve_opt,vswr_max_opt



# Precompute arrays E,F,G,H to speed impedance transformation calculations (lossy TL)
#   zoa     z of first matching section (connected to antenna)
#   zob     z of second matching section
#   lossa   TL loss, dB / radian (zoa section)
#   lossb   TL loss, dB / radian (zob section)
#   flow, fhigh, nfreq      freq band of interest
#   
#
#   Usage:  precompute E,F,G,H once for specified parameters
#       E,F,G,H = series_match_precompute_lossy(zoa=50,zob=75,lossa=0.0,lossb=0.0, nfreq=9,flow=3.5,fhigh=4.0)
#
#       z = (zs*E[a,b] + F[a,b]) / (zs*G[a,b] + H[a,b])
#           where   zs is a row vector of complex zs, shape (1,nfreq)
#                   a,b   lengths of zoa,zob matching section (degrees)
#
@njit
def series_match_precompute_lossy(zoa=50,zob=75,lossa=0.0,lossb=0.0,nfreq=9,flow=3.5,fhigh=4.0):
    # Scale phase delay for freqs across band of interest
    phscale = (np.linspace(flow,fhigh,num=nfreq) / ((fhigh+flow)/2))[None,:]        # shape (1,9)
    E = np.empty((181,181,nfreq), dtype=np.complex128)
    F = np.empty((181,181,nfreq), dtype=np.complex128)
    G = np.empty((181,181,nfreq), dtype=np.complex128)
    H = np.empty((181,181,nfreq), dtype=np.complex128)
    db_per_np = 20*np.log10(np.e)           # dB / neper
    nlossa = lossa / db_per_np              # Loss, np per rad
    nlossb = lossb / db_per_np              # Loss, np per rad
    for a in range(181):
        ra = np.deg2rad(a)
        tanha = np.tanh(((1j)*phscale + nlossa)*ra)
        for b in range(181):
            rb = np.deg2rad(b)
            tanhb = np.tanh(((1j)*phscale + nlossb)*rb)
            E[a,b] = zob*(zoa + zob * tanha * tanhb)
            F[a,b] = zob*(zoa**2 * tanha + zoa*zob*tanhb)
            G[a,b] = zob*tanha + zoa*tanhb
            H[a,b] = zoa*zob + zoa**2 * tanha * tanhb
    return E,F,G,H
# Exec time 48ms




In [77]:
from necutil import nec5_sim_stdio3, plot_vswr_2

#
# Verify the above functions by:
#   --> simulating a simple dipole antenna, 
#        transforming feedpoint impedances using 1000 randomly-selected series matching sections and the above functions
#   --> directly simulating the antenna and matching sections using 'TL' transmission lines in NEC5 design
#   --> verify that results are identical
#


#
# First compute the feedpoint impedances for a simple dipole centered in the 3.5 - 4.0 MHz band (9 frequencies)
#
nec_dipole = """CE Dipole
GW 1 10 0 0 {z} 0 {y} {z} 0.001
GX 100 010
GE 1 0
GD 0 0 0 0 13 0.005 0 0
EX 4 1 1 1 1.0 0.0
FR 0 {f_num} 0 0 {f_min} {f_step}
XQ 0
EN
"""
res = nec5_sim_stdio3([nec_dipole.format(f_num=9, f_min=3.5, f_step=(4.0-3.5)/8, z=40, y=19.8)])
_,zs0 = zip(*res[0][0][0])
zs0 = np.array([zs0])

# array([[62.76 -112.33j  , 64.369 -83.839j , 66.007 -55.282j ,
#         67.694 -26.597j , 69.454  +2.2749j, 71.316 +31.393j ,
#         73.31  +60.817j , 75.472 +90.604j , 77.84 +120.81j  ]])



#
# Now create a file with the same antenna, plus two transmission lines in series:
#
nectmp_tl = """CE Dipole
GW 1 10 0 0 {z} 0 {y} {z} 0.001
GW 2 1 0 0 2 0 0.001 2 0.001
GW 3 1 0 0 1 0 0.001 1 0.001
GX 100 010
GE 1 0
GD 0 0 0 0 13 0.005 0 0
TL 1 -1 2 -1 {zoa} {len_a} 0 0 0 0
TL 2 -1 3 -1 {zob} {len_b} 0 0 0 0
EX 4 3 1 1 1.0 0.0
FR 0 {f_num} 0 0 {f_min} {f_step}
XQ 0
EN
"""

#
# Make a list of random matching section lengths (degrees)
# TL impedances are zoa=50, zob=75
#
tl_lens = np.random.randint(1,180,(1000,2))

#
# Compute transformed z at output of matching sections for the 80M dipole referenced above 
#  (feedpoint z curve = zs0)
#

# Precompute some tables for our freqs and matching section impedances
A,B,C,D = series_match_precompute(zoa=50,zob=75,nfreq=9,flow=3.5,fhigh=4.0)

# Apply the transformations to feedpoint z 'zs0'
# Result: a (1000,9) array of complex impedances
z_computed = np.apply_along_axis(lambda r: ((zs0*A[r[0],r[1]] + B[r[0],r[1]]) / ((1j)*zs0*C[r[0],r[1]] + D[r[0],r[1]]))[0],
                    axis=1, arr=tl_lens)


#
# Now use NEC5 design with two transmission lines to compute z transformations via simulation
#
designs = []
for a, b in tl_lens:
    # Note: must convert from degrees to meters for NEC
    len_a = 299.792458 / 3.75 * a / 360.0
    len_b = 299.792458 / 3.75 * b / 360.0
    designs.append(nectmp_tl.format(f_num=9, f_min=3.5, f_step=(4.0-3.5)/8, z=40, y=19.8, zoa=50, zob=75, len_a=len_a, len_b=len_b))
res = nec5_sim_stdio3(designs)

# Extracts feedpoint complex z for each design --> complex array of shape (#designs, #freqs)
zs = np.array([[freq[1] for freq in des[0][0]] for des in res])


#
# Finally, compare the magnitudes of the two sets of impedances and look for discrepancies
#
mag_err = np.max(np.abs(zs - z_computed), axis=1)
np.max(mag_err)             # --> 0.6837005292713588
np.argmax(mag_err)          # 21
z_computed[21]
# array([525.45007869-242.46725487j, 282.85210649-194.43569841j,
#        193.85354317-116.16487563j, 161.09181286 -51.30791898j,
#        156.32510317  +5.83956685j, 175.37215487 +62.54934557j,
#        230.28303007+123.38294101j, 362.62812643+174.70388346j,
#        633.13178111 +73.15828233j])
zs[21]
# array([525.05-242.78j  , 282.68-194.49j  , 193.78-116.19j  ,
#        161.06 -51.337j , 156.33  +5.8078j, 175.41 +62.514j ,
#        230.38+123.33j  , 362.84+174.55j  , 633.32 +72.501j ])

# ** Result:  Matches to within a very small error


array([10.305+17.273j , 15.183+21.696j , 23.296+25.278j , 35.73 +24.486j ,
       47.972+12.906j , 46.934 -5.8949j, 34.289-15.825j , 22.641-15.915j ,
       15.15 -12.36j  ])

In [4]:
#
# Now repeat with lossy equations, but with loss=0.0
#
E,F,G,H = series_match_precompute_lossy(zoa=50,zob=75,nfreq=9,flow=3.5,fhigh=4.0,lossa=0.0,lossb=0.0)

# Apply the transformations to feedpoint z 'zs0'
# Result: a (1000,9) array of complex impedances
z_computed_lossy = np.apply_along_axis(lambda r: ((zs0*E[r[0],r[1]] + F[r[0],r[1]]) / (zs0*G[r[0],r[1]] + H[r[0],r[1]]))[0],
                    axis=1, arr=tl_lens)

mag_err = np.max(np.abs(zs - z_computed_lossy), axis=1)
np.max(mag_err)             # --> 0.6837005292706405
np.argmax(mag_err)          # 21
z_computed_lossy[21]
# array([525.45007869-242.46725487j, 282.85210649-194.43569841j,
#        193.85354317-116.16487563j, 161.09181286 -51.30791898j,
#        156.32510317  +5.83956685j, 175.37215487 +62.54934557j,
#        230.28303007+123.38294101j, 362.62812643+174.70388346j,
#        633.13178111 +73.15828233j])
zs[21]
# array([525.05-242.78j  , 282.68-194.49j  , 193.78-116.19j  ,
#        161.06 -51.337j , 156.33  +5.8078j, 175.41 +62.514j ,
#        230.38+123.33j  , 362.84+174.55j  , 633.32 +72.501j ])

# ** Result:  Matches to within a very small error



array([15.528+14.749j , 22.815+20.968j , 35.011+26.221j , 54.222+25.467j ,
       74.963 +8.2568j, 76.163-22.986j , 56.565-41.793j , 37.275-43.77j  ,
       24.768-39.177j ])

In [29]:
from necutil import vswr, refl_coef

#
# Check our functions against the published equations for series section matching (Regier 1971).
#

#
# Implements the series-section-matching equations from Regier (1971)
#   zl      feedpoint impedances (numpy array)
#   z0a     z0 of TL nearest antenna (and feedline to radio)
#   z0b     z0 of matching section
#
#   Returns:    electrical lengths of TL sections (degrees):
#       [len(a), len(b), len(a)(second solution), len(b)(second solution)]
#       shape (len(zl), 4)

def ssm_thetas(zl, z0a=50, z0b=75):
    n = z0b/z0a
    r = np.real(zl) / z0a
    x = np.imag(zl) / z0a
    thetab = np.arctan(np.sqrt(((r-1)**2 + x**2) / (r*(n-1/n)**2 - (r-1)**2 - x**2)))
    thetaa = np.arctan(((n-r/n) * np.tan(thetab) + x) / (r + x*n*np.tan(thetab) - 1))
    if not isinstance(zl, np.ndarray):
        if thetaa < 0:
            thetaa += np.pi
    else:
        thetaa[thetaa < 0] += np.pi
    thetabn = np.pi - thetab
    thetaan = np.arctan(((n-r/n) * np.tan(thetabn) + x) / (r + x*n*np.tan(thetabn) - 1))
    if not isinstance(zl, np.ndarray):
        if thetaan < 0:
            thetaan += np.pi
    else:
        thetaan[thetaan < 0] += np.pi
    return np.rad2deg(  np.array((thetaa,thetab,thetaan,thetabn)).transpose() )




In [78]:

# Generate 10000 random complex impedances that meet constraint of SSM equations
ztest = np.random.rand(10000)*100 + (np.random.rand(10000)*200 - 100)*(1j)
# SSM requires VSWR < (75/50)**2, so retain only valid Zs
zvalid = ztest[vswr(ztest) < (75/50)**2]

# Compute the matching section lengths for zvalid (two solutions per z value)
thetas = np.rint(ssm_thetas(zvalid)).astype(int)        # As integer degrees


In [79]:

# Confirm agreement between our impedance-transformation code and the values
#  computed by the SSM equations:
# For each random impedance:
#   * transform using the electrical lengths computed by SSM equations (both solutions)
#   * verify that all values transform back to 50 ohms

A,B,C,D = series_match_precompute(zoa=50,zob=75,nfreq=1,flow=4.0,fhigh=4.0)     # Ignore freqs
z_transformed = np.zeros((zvalid.shape[0],2), dtype=np.complex128)
for i in range(zvalid.shape[0]):
    zs = zvalid[i]
    a = thetas[i,0]
    b = thetas[i,1]
    z_transformed[i,0] = (zs*A[a,b] + B[a,b]) / ((1j)*zs*C[a,b] + D[a,b])[0]
    a = thetas[i,2]
    b = thetas[i,3]
    z_transformed[i,1] = (zs*A[a,b] + B[a,b]) / ((1j)*zs*C[a,b] + D[a,b])[0]

# Print maximum errors in real, imag parts
print(np.max(np.abs(np.real(z_transformed[:,0]) - 50)))
print(np.max(np.abs(np.imag(z_transformed[:,0]) - 0)))
print(np.max(np.abs(np.real(z_transformed[:,1]) - 50)))
print(np.max(np.abs(np.imag(z_transformed[:,1]) - 0)))


0.3715813882810721
1.0125161583320403
0.3630960901532987
1.0735576990618032
